In [1]:
!pip install transformers
!pip install datasets

In [2]:
import sys

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertModel

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, classification_report

from tqdm import tqdm

sys.path.insert(0, '..')
from src.data_collection import get_data

pd.set_option("display.max_colwidth", None)

## Data Preprocessing

In [3]:
dataset = get_data()

Fetching data...


Using custom data configuration ucberkeley-dlab--measuring-hate-speech-7cb9b0b8e4d0e1dd
Reusing dataset parquet (/Users/snehajhaveri/.cache/huggingface/datasets/parquet/ucberkeley-dlab--measuring-hate-speech-7cb9b0b8e4d0e1dd/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/1 [00:00<?, ?it/s]

Processing...
Done!


In [4]:
print(len(dataset))
print(dataset["text"])
print(dataset["hatespeech"])

39565
0                                                                       ! thank u! im transmasc and generally present masc i Maybe look a bit gay so i was wondering about how that would go. im sorry u were heckled but its good to hear things are better there now. tysm my favorite lesbian :)
1                                                                                                                                                                                                                                                                         !Go fuck yourself faggot!
2                                                                                                                                                                                                                                                              !flair [I love women and minorities]
3                                                                                                                     

In [5]:
dataset["hatespeech"].value_counts()

0    26608
1    12957
Name: hatespeech, dtype: int64

In [6]:
device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
MODEL_NAME = "bert-base-uncased"  
BATCH_SIZE = 16
MAX_LEN = 128
EPOCHS = 10
LEARNING_RATE = 1e-05
TOKENIZER = BertTokenizer.from_pretrained(MODEL_NAME, truncation=True, do_lower_case=True)

In [8]:
class Dataset_Preprocess(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = OneHotEncoder(sparse=False).fit_transform(np.array(self.data["hatespeech"]).reshape(-1, 1))
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.targets[index], dtype=torch.float)
        }

In [9]:
# Dataloader creation for dataset 

train_size = 0.8
val_size = 0.1

train_data = dataset.sample(frac = train_size)
test_data = dataset.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)
val_data = test_data.sample(frac=val_size / (1 - train_size), random_state=220).reset_index()
test_data = test_data.drop(val_data.index).reset_index(drop=True)

print(f"Full Dataset Size: {dataset.shape}")
print(f"Train Dataset Size: {train_data.shape}")
print(f"Validation Dataset Size: {val_data.shape}")
print(f"Test Dataset Size: {test_data.shape}")

training_set = Dataset_Preprocess(train_data, TOKENIZER, MAX_LEN)
validation_set = Dataset_Preprocess(val_data, TOKENIZER, MAX_LEN)
testing_set = Dataset_Preprocess(test_data, TOKENIZER, MAX_LEN)

Full Dataset Size: (39565, 2)
Train Dataset Size: (31652, 2)
Validation Dataset Size: (3957, 3)
Test Dataset Size: (3956, 2)


In [ ]:
train_params = {
    "batch_size": BATCH_SIZE,
    "shuffle": True,
    "num_workers": 0
}

val_params = {
    "batch_size": 1,
    "shuffle": False,
    "num_workers": 0
}

test_params = {
    "batch_size": 1,
    "shuffle": False,
    "num_workers": 0
}

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)
testing_loader = DataLoader(testing_set, **test_params)

## BERT Base Model

In [ ]:
import gc
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import AutoModel
from transformers import BertModel
import pandas as pd


In [ ]:
class BERT_Base(nn.Module):
    def __init__(self, n_classes):
        super(BERT_Base, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased')
        self.pre_classifier = nn.Linear(768, 768)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, n_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
# class BERT_CNN(nn.Module):

#     def __init__(self):
#         super(BERT_CNN, self).__init__()
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.conv = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 768), padding='valid')
#         self.relu = nn.ReLU()
#         self.pool = nn.MaxPool2d(kernel_size=(3,1), stride=1)
#         self.dropout = nn.Dropout(0.1)
#         self.fc = nn.Linear(416, 3)
#         self.flat = nn.Flatten()
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, sent_id, mask, token_type_ids):
#         _, all_layers = self.bert(input_ids = sent_id, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
#         # all_layers  = [13, 32, 64, 768]
#         print('all layers', all_layers)
#         print('all layers', all_layers.shape)
#         x = torch.transpose(torch.cat(tuple([t.unsqueeze(0) for t in all_layers]), 0), 0, 1)
#         del all_layers
#         gc.collect()
#         torch.cuda.empty_cache()
#         print('Before dropout',x.shape)
#         x = self.dropout(x)
#         print('After dropout', x.shape)
#         x = self.conv(x)
#         print('After Convolutional layer', x.shape)
#         x = self.pool(self.dropout(self.relu(x)))
#         x = self.fc(self.dropout(self.flat(self.dropout(x))))
#         return self.softmax(x)

In [ ]:
num_classes = dataset["hatespeech"].nunique()
model = BERT_Base(n_classes = num_classes)
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT_Base(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

## Model Training

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    model.train()
    for _, data in tqdm(enumerate(training_loader, 0)):
        ids = data["ids"].to(device, dtype=torch.long)
        mask = data["mask"].to(device, dtype=torch.long)
        token_type_ids = data["token_type_ids"].to(device, dtype=torch.long)
        targets = data["targets"].to(device, dtype=torch.float)
        # print('ids', type(ids))
        # print('mask', type(mask))
        # print('token type ids', type(token_type_ids))
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _ % 1000 == 0:
            print(f"Epoch: {epoch}, Loss: {loss.item()}")
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss: 0.7021530270576477


1000it [05:45,  2.80it/s]

Epoch: 0, Loss: 0.5504308342933655


1979it [11:34,  2.85it/s]
0it [00:00, ?it/s]

...

1979it [11:47,  2.80it/s]
0it [00:00, ?it/s]

Epoch: 2, Loss: 0.19588738679885864


1000it [05:57,  2.81it/s]

Epoch: 2, Loss: 0.1955200880765915


1979it [11:46,  2.80it/s]


## Model Evaluation

In [ ]:
def validation(model, loader):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(loader, 0)):
            ids = data["ids"].to(device, dtype=torch.long)
            mask = data["mask"].to(device, dtype=torch.long)
            token_type_ids = data["token_type_ids"].to(device, dtype=torch.long)
            targets = data["targets"].to(device, dtype=torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation(model, validation_loader)

final_outputs = np.argmax(outputs, axis=1)
targets = np.argmax(targets, axis=1)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
3957it [00:45, 86.77it/s]


In [ ]:
print(f"Got {sum(final_outputs == targets)} / {len(final_outputs)} correct")

Got 3037 / 3957 correct


In [ ]:
micro_f1 = f1_score(targets, final_outputs, average="micro")
macro_f1 = f1_score(targets, final_outputs, average="macro")
weighted_f1 = f1_score(targets, final_outputs, average="weighted")

print(f"Micro F1 score:\t\t{round(micro_f1, 3)}")
print(f"Macro F1 score:\t\t{round(macro_f1, 3)}")
print(f"Weighted F1 score:\t{round(weighted_f1, 3)}")

Micro F1 score:		0.768
Macro F1 score:		0.747
Weighted F1 score:	0.772


In [ ]:
print(classification_report(targets, final_outputs))

              precision    recall  f1-score   support

           0       0.86      0.78      0.82      2665
           1       0.62      0.74      0.68      1292

    accuracy                           0.77      3957
   macro avg       0.74      0.76      0.75      3957
weighted avg       0.78      0.77      0.77      3957



In [ ]:
output_model_file = "../models/pytorch_bert_cnn.bin"
output_vocab_file = "../models/vocab_bert_cnn.bin"

torch.save(model, output_model_file)
TOKENIZER.save_vocabulary(output_vocab_file)

print("Model Saved")